In [1]:
import os
os.chdir('..')

In [2]:
import tensorflow as tf
from tensorflow.python.ops import variable_scope
from tensorflow.python.ops import array_ops
from tensorflow.python.framework import ops
import tensorflow.keras as keras
# Source: http://vict0rsch.github.io/2018/06/06/tensorflow-streaming-multilabel-f1/

/home/matt/tools/miniconda3/envs/kaggle-hp/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/matt/tools/miniconda3/envs/kaggle-hp/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


### Keras
Can we define the custom metric in Keras?

In [4]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Activation, Conv2D, Flatten
from tensorflow.keras.layers import MaxPooling2D
def build_model():
    """Builds the testModel1 Architecture.
    """
    inputs = Input(shape=(264, 264, 4))
    x = Conv2D(64, (3, 3), activation=tf.nn.relu)(inputs)
    x = MaxPooling2D((2, 2))(x)

    for i in range(2):
        x = Conv2D(64, (3, 3), activation=tf.nn.relu)(x)
        x = MaxPooling2D((2, 2))(x)

    x = Flatten()(x)
    x = Dense(264, activation=tf.nn.relu)(x)
    outputs = Dense(28, activation=tf.nn.sigmoid)(x)

    model = keras.Model(inputs=inputs, outputs=outputs)

    optimizer = tf.train.AdamOptimizer(0.001)

    model.compile(optimizer=optimizer,
                loss='binary_crossentropy',
                metrics={'f1_macro' : f1_macro})
    
    return model

In [3]:
import tensorflow.keras.backend as K
from sklearn.metrics import f1_score

def f1_macro(y_true, y_pred):
    
    y_true = K.cast(y_true, tf.float64)
    y_pred = K.cast(y_pred, tf.float64)
    
    TP = K.sum(y_true * K.round(y_pred), axis=0)
    FN = K.sum(y_true * (1 - K.round(y_pred)), axis=0)
    FP = K.sum((1 - y_true) * K.round(y_pred), axis=0)
    
    prec = TP / (TP + FP)
    rec = TP / (TP + FN)
    
    # Convert NaNs to Zero
    prec = tf.where(tf.is_nan(prec), tf.zeros_like(prec), prec)
    rec = tf.where(tf.is_nan(rec), tf.zeros_like(rec), rec)
    
    f1 = 2 * (prec * rec) / (prec + rec)
    
    # Convert NaN to Zero
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    f1 = K.mean(f1)
    
    return f1

In [5]:
model = build_model()

In [6]:
import numpy as np
random_data_train = np.random.rand(100, 264, 264, 4)
val_data_train = np.random.rand(100, 264, 264, 4)


true_labels_train = np.round(np.random.rand(100, 28))
val_label_train = np.round(np.random.rand(100, 28))

In [7]:
history = model.fit(random_data_train, true_labels_train, validation_data=(val_data_train, val_label_train))

Train on 100 samples, validate on 100 samples
Epoch 1/1
100/100 [==============================] - 5s 47ms/step - loss: 1.0982 - val_loss: 0.7115


In [188]:
history.history

{'val_loss': [0.6931105804443359], 'loss': [0.6939537143707275]}

In [9]:
model.metrics

{'f1_macro': <function __main__.f1_macro(y_true, y_pred)>}

In [168]:
rounded_preds = np.round(preds)

In [169]:
rounded_preds

array([[0., 1., 0., ..., 0., 0., 1.],
       [1., 1., 0., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 1.],
       ...,
       [0., 1., 0., ..., 0., 0., 1.],
       [0., 1., 0., ..., 0., 0., 0.],
       [1., 1., 0., ..., 0., 0., 1.]], dtype=float32)

In [170]:
f1_score(true_labels_train, rounded_preds, average='macro')

/home/matt/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


0.36293459832079195

In [171]:
f1 = f1_macro(true_labels_train, rounded_preds)

In [172]:
K.eval(f1)

0.36293459832079183

In [114]:
from sklearn.metrics import precision_score, recall_score

In [117]:
print(precision_score(true_labels_train, rounded_preds, average='macro'))
print(recall_score(true_labels_train, rounded_preds, average='macro'))

0.26139455782312926
0.550595238095238


/home/matt/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [74]:

K.eval(TP / (TP + FP))

array([1.       , 1.       , 0.6666667, 1.       , 0.       , 0.       ,
       1.       , 0.       , 1.       , 0.75     , 0.       , 1.       ,
       0.       , 0.       , 1.       , 1.       , 0.       , 1.       ,
       0.       , 1.       , 1.       , 0.       , 1.       , 1.       ,
       0.       , 0.       , 0.       , 1.       ], dtype=float32)

In [9]:
estimator = keras.estimator.model_to_estimator(model)

INFO:tensorflow:Using the Keras model provided.
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp2cfthip0', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fe19aceeb38>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [9]:
model.

{}